# 0. Importar Librerías

### Imports

In [1]:
import supervision as sv
from roboflow import Roboflow

import os
import cv2
import glob
import copy
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from imutils.video import FileVideoStream

from sinks.model_sink import ModelSink
from sinks.annotation_sink import AnnotationSink

from tools.video_info import VideoInfo
from tools.write_data import txt_append, write_txt

import roboflow_config


from icecream import ic

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


### Variables

En el primer ciclo, se toma el video y se procesa con el modelo base del detector, pero en este caso se usa el modelo **yolov8x_cf_v5.pt**, modelo que tiene las clases "person, bus, bicycle, motorcycle, truck, car". Con esto, se obtiene el primer CSV del video, y es el que se va a analizar. Si no se cuentan con modelos entrenados, se utilizaría un modelo pre-entrenado (Ej. **yolov8x.pt**), teniendo en cuenta que estos modelos están entrenados para detectar las 80 clases del dataset Microsoft COCO.

In [2]:
ROOT = Path('D:/Data/MARZO_2024')
SOURCE_FOLDER = ROOT / 'Videos' / 'ARS_C57_C65'
INPUT_VIDEO = 'ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new.mp4'
SOURCE_CSV = ROOT / 'Detections' / 'ARS_C57_C65'
INPUT_CSV = 'ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new_yolov10.csv'

SOURCES_FOLDER = ROOT / 'Videos'
DETECTIONS_FOLDER = ROOT / 'Detections'
DATASET_FOLDER = ROOT / 'dataset'

### Obtener todos los archivos CSV

In [3]:
csv_list = []
for root, dirs, files in os.walk(Path(DETECTIONS_FOLDER)):
    for file in files:
        if file.endswith('v10.csv'):
            csv_list.append(Path(root) / file)
csv_list

[WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM2_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM3_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM4_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_Iguana/ARS_Iguana_CAM1_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_Iguana/ARS_Iguana_CAM2_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_Iguana/ARS_Iguana_CAM3_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_SanGerman/ARS_San_German_CAM1_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_SanGerman/ARS_San_German_CAM2_new_yolov10.csv'),
 WindowsPath('D:/Data/MARZO_2024/Detections/ARS_SanPedro/ARS_Metroplus_San_Pedro

In [4]:
mp4_list = []
for root, dirs, files in os.walk(Path(SOURCES_FOLDER)):
    for file in files:
        if file.endswith('.mp4'):
            mp4_list.append(Path(root) / file)
mp4_list

[WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM2_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM3_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM4_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_Iguana/ARS_Iguana_CAM1_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_Iguana/ARS_Iguana_CAM2_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_Iguana/ARS_Iguana_CAM3_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_SanGerman/ARS_San_German_CAM1_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_SanGerman/ARS_San_German_CAM2_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_SanPedro/ARS_Metroplus_San_Pedro_CAM1_new.mp4'),
 WindowsPath('D:/Data/MARZO_2024/Videos/ARS_SanPedro/ARS_Metroplus_San_Pedro_CAM2_new.mp4'),
 W

In [6]:
with open(csv_list[0], 'r') as csv_file:
    csv_data = pd.read_csv(
        filepath_or_buffer=csv_file,
        sep=',',
        names=['frame', 'id', 'class', 'x', 'y', 'w', 'h', 'score'],
        header=None,
        index_col=False
    )
data_analysis = csv_data.groupby('frame').agg(detection_number=('frame', 'size'),classes=('class', lambda x: list(x.unique())),class_number=('class',lambda x: len(list(x.unique()))),mean_score=('score', 'mean')).reset_index()
data_classfilter = data_analysis[data_analysis['classes'].apply(lambda x: 'person' in x or 'bicycle' in x)]
data_classfilter
# data_underscores = data_classfilter.sort_values(['class_number', 'detection_number', 'mean_score'], ascending=[False, False, True])
# selected_frame = data_underscores.head(1)
# number = selected_frame['frame'].to_list()

,frame,detection_number,classes,class_number,mean_score
0,0,17,"[bus, person, car, truck]",4,0.525420
1,1,17,"[bus, person, car, truck]",4,0.540096
2,2,18,"[bus, person, car, truck, bicycle]",5,0.532540
3,3,18,"[person, car, bus, truck, motorcycle]",5,0.525020
4,4,19,"[car, person, truck, bus, motorcycle]",5,0.498484
...,...,...,...,...,...
44268,44268,15,"[car, person, bus, motorcycle]",4,0.451231
44269,44269,15,"[car, person, bus, motorcycle]",4,0.455761
44270,44270,16,"[car, person, bus, motorcycle]",4,0.428436
44271,44271,18,"[car, person, bus, motorcycle]",4,0.392288


### Procesar el archivo y obtener el mejor frame

In [20]:
frame_list = []
for data_file, video_file in zip(csv_list, mp4_list):
    with open(data_file, 'r') as csv_file:
        csv_data = pd.read_csv(
            filepath_or_buffer=csv_file,
            sep=',',
            names=['frame', 'id', 'class', 'x', 'y', 'w', 'h', 'score'],
            header=None,
            index_col=False
        )
    data_analysis = csv_data.groupby('frame').agg(detection_number=('frame', 'size'),classes=('class', lambda x: list(x.unique())),class_number=('class',lambda x: len(list(x.unique()))),mean_score=('score', 'mean')).reset_index()
    data_classfilter = data_analysis[data_analysis['classes'].apply(lambda x: 'person' in x or 'bicycle' in x)]
    data_underscores = data_classfilter.sort_values(['class_number', 'detection_number', 'mean_score'], ascending=[False, False, True])
    selected_frame = data_underscores.head(1)
    number = selected_frame['frame'].to_list()
    frame_list.append([data_file, video_file, number[0]])
frame_list

[[WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new_yolov10.csv'),
  WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new.mp4'),
  32689],
 [WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM2_new_yolov10.csv'),
  WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM2_new.mp4'),
  5322],
 [WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM3_new_yolov10.csv'),
  WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM3_new.mp4'),
  2320],
 [WindowsPath('D:/Data/MARZO_2024/Detections/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM4_new_yolov10.csv'),
  WindowsPath('D:/Data/MARZO_2024/Videos/ARS_C57_C65/ARS_Iglesia_Santa_Maria_Magdalena_CAM4_new.mp4'),
  12192],
 [WindowsPath('D:/Data/MARZO_2024/Detections/ARS_Iguana/ARS_Iguana_CAM1_new_yolov10.csv'),

### Captura de frames

In [32]:
os.mkdir(Path(f"{DATASET_FOLDER}"))
os.mkdir(Path(f"{DATASET_FOLDER}/labels"))
os.mkdir(Path(f"{DATASET_FOLDER}/images"))
os.mkdir(Path(f"{DATASET_FOLDER}/detections"))
image_name_pattern: str = "image_{:07d}"
weights = 'D:/Data/models/yolov10/yolov10x.pt'
detection_sink = ModelSink(
    weights_path=weights,
    confidence=0.1,
    class_filter=[0,1,2,3,5,7]
)
source_info = VideoInfo(width=1920,height=1080)
annotation_sink = AnnotationSink(
    source_info=source_info,
    fps=False,
)
video_count = 0
for _, source_video, selected_frame in frame_list:
    print(f"Video: {source_video}, Selected Frame: {selected_frame}")
    frame_number = 0
    video_stream = FileVideoStream(str(Path(source_video)))
    video_stream.start()
    while video_stream.more():
        print(f"Frame: {frame_number}", end='\r', flush=True)
        output_data = []
        image = video_stream.read()
        if image is None:
            print()
            break
        
        if frame_number == selected_frame:
            annotated_image = image.copy()

            results = detection_sink.detect(scene=annotated_image)

            output_data = txt_append(output_data, results)

            detections = sv.Detections.from_ultralytics(results)

            annotated_image = annotation_sink.on_detections(detections=detections, scene=annotated_image)

            write_txt(f"{DATASET_FOLDER}/labels/{video_count}{image_name_pattern.format(frame_number)}.txt", output_data)
            cv2.imwrite(f"{DATASET_FOLDER}/images/{video_count}{image_name_pattern.format(frame_number)}.png", image)
            cv2.imwrite(f"{DATASET_FOLDER}/detections/{video_count}{image_name_pattern.format(frame_number)}.png", annotated_image)
            print(f"Video {video_count}, Frame {frame_number}")
            break
            
        frame_number += 1
    video_stream.stop()
    video_count += 1



Video: D:\Data\MARZO_2024\Videos\ARS_C57_C65\ARS_Iglesia_Santa_Maria_Magdalena_CAM1_new.mp4, Selected Frame: 32689
Video 0, Frame 32689
Video: D:\Data\MARZO_2024\Videos\ARS_C57_C65\ARS_Iglesia_Santa_Maria_Magdalena_CAM2_new.mp4, Selected Frame: 5322
Video 1, Frame 5322
Video: D:\Data\MARZO_2024\Videos\ARS_C57_C65\ARS_Iglesia_Santa_Maria_Magdalena_CAM3_new.mp4, Selected Frame: 2320
Video 2, Frame 2320
Video: D:\Data\MARZO_2024\Videos\ARS_C57_C65\ARS_Iglesia_Santa_Maria_Magdalena_CAM4_new.mp4, Selected Frame: 12192
Video 3, Frame 12192
Video: D:\Data\MARZO_2024\Videos\ARS_Iguana\ARS_Iguana_CAM1_new.mp4, Selected Frame: 24365
Video 4, Frame 24365
Video: D:\Data\MARZO_2024\Videos\ARS_Iguana\ARS_Iguana_CAM2_new.mp4, Selected Frame: 10616
Video 5, Frame 10616
Video: D:\Data\MARZO_2024\Videos\ARS_Iguana\ARS_Iguana_CAM3_new.mp4, Selected Frame: 4919
Video: D:\Data\MARZO_2024\Videos\ARS_SanGerman\ARS_San_German_CAM1_new.mp4, Selected Frame: 23005
Video 7, Frame 23005
Video: D:\Data\MARZO_2024\V

### Construir labelmap

In [33]:
dataset_names= {
    0: 'bicycle',
    1: 'bus',
    2: 'car',
    3: 'motorbike',
    4: 'person',
    5: 'truck'
}
for values in dataset_names.values():
    with open(f"{DATASET_FOLDER}/labelmap.txt", 'a') as txt_file:
        txt_file.write(f"{values}\n")

### Transferir a Roboflow

In [35]:
rf = Roboflow(api_key=roboflow_config.API_KEY)
print(rf.workspace())

loading Roboflow workspace...
{
  "name": "Carlos Andr\u00e9s Wilches P\u00e9rez",
  "url": "carlos-andres-wilches-perez",
  "projects": [
    "carlos-andres-wilches-perez/camaras_fijas",
    "carlos-andres-wilches-perez/cct-drone",
    "carlos-andres-wilches-perez/elementos-de-proteccion-personal-id6zs",
    "carlos-andres-wilches-perez/tejo-tracking",
    "carlos-andres-wilches-perez/vehiculos_2024",
    "carlos-andres-wilches-perez/videos_2024"
  ]
}


### Configuración del dataset

In [36]:
PROJECT_ID = 'vehiculos_2024'
project = rf.workspace().project(PROJECT_ID)
print(project)

loading Roboflow workspace...
loading Roboflow project...
{
  "name": "vehiculos_2024",
  "type": "object-detection",
  "workspace": "carlos-andres-wilches-perez"
}


In [37]:
images = os.listdir(Path(f"{DATASET_FOLDER}/images"))
for image in images:
    print(Path(image).stem)
    project.single_upload(
        batch_name='video1',
        image_path=f"{DATASET_FOLDER}/images/{Path(image).stem}.png",
        annotation_path=f"{DATASET_FOLDER}/labels/{Path(image).stem}.txt",
        annotation_labelmap=f"{DATASET_FOLDER}/labelmap.txt",
    )

0image_0032689
10image_0025284
11image_0011613
12image_0000660
13image_0015184
14image_0010642
15image_0004900
16image_0024957
17image_0015438
18image_0003543
19image_0009722
1image_0005322
20image_0018919
21image_0007945
22image_0009902
23image_0014382
24image_0009956
25image_0000292
26image_0002853
27image_0014867
28image_0019301
29image_0003709
2image_0002320
3image_0012192
4image_0024365
5image_0010616
7image_0023005
8image_0008561
9image_0006857
